sk-1a28c3fcc7e044cbacd6faf47dc89755

In [ ]:
api_key = "sk-1a28c3fcc7e044cbacd6faf47dc89755"

In [5]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI

# Function to fetch and parse the webpage
def fetch_website_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract text from paragraphs or other relevant elements
    paragraphs = soup.find_all('p')
    content = " ".join([p.get_text() for p in paragraphs])
    
    return content

# Fetch the content from the target URL
url = "https://lilianweng.github.io/"
website_content = fetch_website_content(url)

# Initialize the OpenAI client
api_key = "sk-1a28c3fcc7e044cbacd6faf47dc89755"
client = OpenAI(
    api_key=api_key,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# Pass the website content to the OpenAI model for further processing
completion = client.chat.completions.create(
    model="qwen-plus",  # Example model
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': f"Here is some content from the website: {website_content}. How can I help you with it?"}
    ],
    extra_body={
        "enable_search": True
    }
)

# Print the result
print(completion.model_dump_json())

{"id":"chatcmpl-a743a983-4d4b-94ec-9879-26d79a6562d0","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"It looks like you've provided a rich set of content from a blog written by Lilian, covering various topics related to machine learning, reinforcement learning, large language models (LLMs), and more. Based on this content, it seems like you're interested in discussing or summarizing key points from these posts. How can I assist you with this material? Here are a few ways I can help:\n\n1. **Summarize Key Points**: Provide a concise summary of the main ideas discussed in the blog.\n2. **Clarify Concepts**: Explain specific concepts in more detail, such as reward hacking, hallucination, diffusion models, or prompt engineering.\n3. **Expand on a Topic**: Dive deeper into one of the topics mentioned, providing additional context or examples.\n4. **Compare/Contrast**: Compare different techniques or approaches mentioned, such as data augmentation vs. gener

In [ ]:
import streamlit as st
import requests
from bs4 import BeautifulSoup
from openai import OpenAI

# Function to fetch and parse the webpage
def fetch_website_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract text from paragraphs or other relevant elements
        paragraphs = soup.find_all('p')
        content = " ".join([p.get_text() for p in paragraphs])
        return content
    except Exception as e:
        return f"Error fetching content: {e}"

# Streamlit UI
st.title("Website Content Analyzer")

# Input URL
url = st.text_input("Enter the website URL:", "https://lilianweng.github.io/")

if st.button("Analyze Website"):
    # Fetch content from the provided URL
    website_content = fetch_website_content(url)

    if website_content.startswith("Error"):
        st.error(website_content)
    else:
        # Initialize the OpenAI client
        api_key = "sk-1a28c3fcc7e044cbacd6faf47dc89755"
        client = OpenAI(
            api_key=api_key,
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
        )

        # Pass the website content to the OpenAI model for further processing
        completion = client.chat.completions.create(
            model="qwen-plus",  # Example model
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': f"Here is some content from the website: {website_content}. How can I help you with it?"}
            ],
            extra_body={
                "enable_search": True
            }
        )

        # Display result
        st.subheader("Response from AI")
        st.json(completion.model_dump_json())

In [7]:
import streamlit as st
import requests
import os
import dashscope
from bs4 import BeautifulSoup

# Function to fetch and parse the webpage
def fetch_website_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors
        soup = BeautifulSoup(response.content, "html.parser")

        # Find the latest topics by extracting titles and summaries
        # For simplicity, assuming articles are in <article> or <h2> tags. Adjust as needed.
        articles = soup.find_all(['h2', 'article'])

        content = []
        for idx, article in enumerate(articles[:10]):  # Only fetch the latest 10 articles
            title = article.get_text().strip()
            one_liner = article.find_next('p').get_text().strip() if article.find_next('p') else "No summary available"
            content.append(f"{idx + 1}. \"{title}\", \"{one_liner}\", {url}")
        
        return content
    except Exception as e:
        return f"Error fetching content: {e}"

# Function to summarize the content using LLM
def summarize_content(content):
    api_key = "sk-1a28c3fcc7e044cbacd6faf47dc89755"  # Replace with your actual API key
    # Prepare the messages for summarization
    messages = [
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': f"Please summarize the following content into the latest 10 key ideas in the format: '1. title, one-liner description, website name': {content}"}
    ]
    
    # Call the Dashscope API to summarize the content
    try:
        response = dashscope.Generation.call(
            api_key=api_key,
            model="qwen-plus",  # Example model, replace as needed
            messages=messages,
            enable_search=True,
            result_format='message'
        )

        # Access the response content correctly
        summarized_content = response['message']
        return summarized_content

    except Exception as e:
        print(f"Error: {e}")  # Catch any other errors
        return f"Error summarizing content: {e}"

# Streamlit UI
st.title("Website Content Analyzer and Summarizer")

# URL selection
st.sidebar.header("Select Websites to Analyze")
website_choices = [
    "https://www.qbitai.com/",
    "https://www.jiqizhixin.com/",
    "https://lilianweng.github.io/",
    "https://x.com/deepseek_ai"
]
selected_websites = st.sidebar.multiselect(
    "Choose websites to analyze:",
    website_choices,
    default=website_choices  # Default to analyzing all websites
)

if st.button("Analyze Websites"):
    # Loop through selected websites and fetch content
    all_content = []
    for website in selected_websites:
        st.subheader(f"Latest Topics from {website}")
        website_content = fetch_website_content(website)
        
        if isinstance(website_content, str) and website_content.startswith("Error"):
            st.error(website_content)
        else:
            for topic in website_content:
                st.write(topic)
            all_content.append("\n".join(website_content))
    
    # Combine all content from the websites
    combined_content = "\n\n".join(all_content)
    
    # Summarize the content using LLM
    st.subheader("Summarized Key Ideas from Each Website")
    summarized_content = summarize_content(combined_content)
    st.write(summarized_content)

2025-02-19 16:10:21.313 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 16:10:21.350 
  command:

    streamlit run /Users/chengyixu/Library/CloudStorage/OneDrive-Personal/主业/Minerva/Minerva Agent/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-02-19 16:10:21.351 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 16:10:21.351 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 16:10:21.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 16:10:21.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 16:10:21.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored w

In [ ]:
import dashscope
import os
import datetime  # Import datetime module

def get_website_topics(domain):
    messages = [
        {'role': 'system', 'content': 'You are a web researcher. Format responses as "1. Title | Description | Website"'},
        {'role': 'user', 'content': f'''
        Search and list the latest 10 topics from {domain} with:
        1. Article titles
        2. One-line descriptions
        3. Website name
        Use current date: {datetime.date.today()}'''}
    ]
    
    response = dashscope.Generation.call(
        api_key = "sk-1a28c3fcc7e044cbacd6faf47dc89755",
        model="qwen-max",  # Newest model for better search integration
        messages=messages,
        enable_search=True,
        result_format='message'
    )
    return response['output']['choices'][0]['message']['content']

websites = [
    "qbitai.com",
    "jiqizhixin.com",
    "lilianweng.github.io",
    "x.com/deepseek_ai"
]

for site in websites:
    print(f"Latest from {site}:")
    print(get_website_topics(site))
    print("\n"+"-"*50+"\n")

Latest from qbitai.com:
I'm unable to directly browse the web or access real-time data, including the latest articles from qbitai.com as of 2025-02-19. However, I can guide you on how to find this information yourself or provide a hypothetical example based on typical content from the site. For actual and current results, you might want to visit qbitai.com directly and check their latest posts. If you need a mock-up of what such a list might look like, let me know!

--------------------------------------------------

Latest from jiqizhixin.com:
I'm unable to directly access or browse the internet in real-time, including specific dates in the future, such as 2025-02-19. However, I can guide you on how to find the latest articles from jiqizhixin.com yourself. You can visit the website and look for the most recent posts, which are typically listed on the homepage or in a dedicated "Latest" section. For the most accurate and up-to-date information, please check the site directly. If you ne

In [19]:
import requests
import datetime  # Import datetime module
import dashscope

# Function to get raw HTML content from the websites
def get_raw_html(domain):
    try:
        # Send request to the website and get the raw HTML content
        response = requests.get(f'http://{domain}')
        if response.status_code == 200:
            return response.content
        else:
            return f"Failed to retrieve content from {domain}. HTTP Status Code: {response.status_code}"
    except Exception as e:
        return f"Error while fetching content from {domain}: {str(e)}"

# Function to prepare the message for Qwen LLM analysis
def analyze_with_qwen(domain, raw_html):
    messages = [
        {'role': 'system', 'content': 'You are a web researcher. Analyze the raw HTML content and extract key topics in the following format: "1. Title | Description | Website"'},
        {'role': 'user', 'content': f'''
        Analyze the raw HTML content from {domain} and provide the latest 10 topics with:
        1. Article titles in English
        2. Article titles in Chinese
        3. One-line descriptions in English
        4. One-line descriptions in Chinese
        5. Website name
        Use current date: {datetime.date.today()}.
        HTML Content: {raw_html.decode('utf-8')}
        '''}
    ]

    response = dashscope.Generation.call(
        api_key="sk-1a28c3fcc7e044cbacd6faf47dc89755",
        model="qwen-max",
        messages=messages,
        enable_search=True,
        result_format='message'
    )
    return response['output']['choices'][0]['message']['content']

# List of websites to scrape
websites = [
    "qbitai.com",
    "jiqizhixin.com",
    "lilianweng.github.io",
    "x.com/deepseek_ai"
]

# Scraping and analyzing websites
for site in websites:
    print(f"Scraping raw HTML from {site}...")
    raw_html = get_raw_html(site)
    
    # Check if there was an error
    if isinstance(raw_html, str) and ('Error' in raw_html or 'Failed' in raw_html):
        print(raw_html)
    else:
        print(f"Raw HTML retrieved from {site}. Analyzing with Qwen LLM...\n")
        qwen_analysis = analyze_with_qwen(site, raw_html)
        print(f"Analysis from {site}:\n{qwen_analysis}")
    
    print("\n" + "-" * 50 + "\n")

Scraping raw HTML from qbitai.com...
Failed to retrieve content from qbitai.com. HTTP Status Code: 403

--------------------------------------------------

Scraping raw HTML from jiqizhixin.com...
Failed to retrieve content from jiqizhixin.com. HTTP Status Code: 403

--------------------------------------------------

Scraping raw HTML from lilianweng.github.io...
Raw HTML retrieved from lilianweng.github.io. Analyzing with Qwen LLM...



ProxyError: HTTPSConnectionPool(host='dashscope.aliyuncs.com', port=443): Max retries exceeded with url: /api/v1/services/aigc/text-generation/generation (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))